# Plonk Implementation(Python Version)

## Problem Definition

We take **Square-Fibonacci** as an example to demonstrate the process of proof generation

Defination of Square-Fibonacci Problem:
- Let $f_0 = 1, f_1 = 1$
- For $i \ge 2$, define $f_i:=(f_{i-2})^2+(f_{i-1})^2 \ mod \ q$
    - $q$ is a large prime integer, used to bound the size of each element, so that it can be represented by some predetermined number of bits.

Let $n$ be some very large integer. For convenience, we assume $n$ is a power of 2

Let $k$ be the $n^{th}$ Square-Fibonacci number

**Our goal**: generate an efficiently-verifiable proof $\pi$ showing that indeed $k$ is the $n^{th}$ Square-Fibonacci number(i.e. $f_n = k$)

## Phases of Proof Generation
The Plonk-based proof generation consists of 3 steps:
1. Filling in the trace table
2. Committing to the trace table
3. Proving the trace table's correctness

In [1]:
from sage import *

# some basic statement
q = 0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabfffeb153ffffb9feffffffffaaab
F_q = GF(q)
n = 8
k = 317754178345286893212434
f_0 = F_q(1)
f_1 = F_q(1)

## Step 1: Filling in the Trace Table
The trace table is a 2-dimensional matrix where 'witness' or 'trace' is written down, that is (n rows * 5 cols)
- 5 columns:
    - $A, B, C$: represent witness data / private input, each row lists 3 sequential Sequare-Fibonacci numbers
        - e.g. the $i^{th}$ row $(f_i, f_{i+1}, f_{i+2})$ is a witness for $(i+2)^{th}$ Sequare-Fibonacci number
    - $S$: represents selector column, indicating a certain mathmatical relation should hold over the element of the row.
        - $1$ represents the first 3 elements of the row $(a, b, c)$ must satisfy $c = a^2 + b^2 \ mod \ q$
        - $0$ represents the condition does not need to be satisfied.
    - $P$: represents public inputs, inputs to the circuit that are public known. 
        - e.g. the first two values of the sequence $f_0, f_1$ and $k$ as the value to be proved
- n rows: left a blank row, so that the height of the table becomes $n$, an even power of 2
    - $1^{st}$ row: $f_0, f_1, f_2, 1, f_0$
    - $2^{nd}$ row: $f_1, f_2, f_3, 1, f_1$
    - $3^{rd}$ row: $f_2, f_3, f_4, 1, k$
    - ...
    - $(n-2)^{th}$ row: $f_{n-2}, f_{n-1}, f_n, 1, "" $
    - $(n-1)^{th}$ row: $"", "", "", 0, ""$

Next step is to fill in the trace table: either copy or compute over $F_p$

In [2]:
# generate witness/fill in the trace table
def witness_generation(f_0, f_1, k, n):
    
    trace_table = []
    
    # init col A, B, C, S
    f_a = f_0
    f_b = f_1
    f_c = f_b
    S = F_q(1)
    
    for i in range(n-1):
        f_a = f_b
        f_b = f_c
        f_c = f_a**2 + f_b**2
        trace_table.append([f_a, f_b, f_c, S,""])
        
    # add a blank row to get n row
    S = 0
    trace_table.append(["","","", S,""])
    
    # add public parameters
    trace_table[0][4] = f_0
    trace_table[1][4] = f_1
    trace_table[2][4] = k
    
    return trace_table

trace_table = witness_generation(f_0, f_1, k, n)
print(f"trace table is {trace_table}")

trace table is [[1, 1, 2, 1, 1], [1, 2, 5, 1, 1], [2, 5, 29, 1, 317754178345286893212434], [5, 29, 866, 1, ''], [29, 866, 750797, 1, ''], [866, 750797, 563696885165, 1, ''], [750797, 563696885165, 317754178345286893212434, 1, ''], ['', '', '', 0, '']]


## Step 2: Commit to the Trace Table

### interpret the trace table columns as polynomials

Each column can be considered as a length-$n$ vector of finite field elements $\rightarrow$ this vector can be regarded as the evaluation form of a polynomial $A(x)$ with degree $(n-1)$: the $i^{th}$ element of $A$ corresponds to the evaluation $A(\omega^i)$, where $\omega \in F_q$ is **$n^{th}$ root of unity** and has order $n$

### Commit to column polynomials
Now that we have known how to interpret the columns as polynomials, we can commit to each of them using a polynomial commitment scheme.

In [3]:
# find a subgroup whose order = n
# def find_nth_root_of_unity(F_q, n):
    
#     while True:
#         w = F_q.random_element()
#         if w.multiplicative_order() == n:
#             break
#         else:
#             print(f'w = {w}, whose order is {w.multiplicative_order()}, not statified')
#     return w

# w = find_nth_root_of_unity(F_q, n)
# print(w)

# find a group: {1,2,4,7,8,11,13,14}
w_list = [1,2,4,7,8,11,13,14]